In [1]:
import jax.numpy as jnp
import jax
FLATFORM = 'gcp'
if FLATFORM=='kaggle':
    IsTPU = True
    dtype = jnp.bfloat16
    num_workers = 16
    batch_size = 12 * 8
#     try:
#         import transformers
#         speech_path = 'bengaliai-speech/train_mp3s/'
#         data_path = 'bengaliai-speech/train.csv'
#         output_path = ''
#     except:
    !pip install librosa --quiet  --upgrade
    !pip install git+https://github.com/zhenlan0426/transformers.git --quiet  --upgrade
    #!pip install evaluate --quiet  --upgrade
    !pip install sentencepiece --quiet  --upgrade
    #!pip install jiwer --quiet  --upgrade
    !pip install audiomentations --quiet  --upgrade
    speech_path = '/kaggle/input/bengaliai-speech/train_mp3s/'
    data_path = '/kaggle/input/bengaliai-speech/train.csv'
    output_path = '/kaggle/working/'
    background = '/kaggle/input/esc-50/audio'
    model_path = '/kaggle/input/model-add-data-all/model_all'
    add_data_path = '/kaggle/input/dlsprint/train.csv'
    add_audio = '/kaggle/input/dlsprint/train_files/'
elif FLATFORM=='gcp':
    #!pip install tensorrt --upgrade
    #%pip install git+https://github.com/zhenlan0426/transformers.git  --upgrade
    IsTPU = True
    dtype = jnp.bfloat16
    num_workers = 16
    batch_size = 12 * 8
    speech_path = 'data/train_mp3s/'
    data_path = 'data/train.csv'
    output_path = ''
    background = 'background/audio/'
    model_path = 'model_all'
    add_data_path = 'dlsprint/train.csv'
    add_audio = 'dlsprint/train_files/'
    add_data_path2 = 'text'
    add_audio2 = '/mnt/disks/persist/RESPIN/'
else:
    IsTPU = False
    batch_size = 32
    dtype = jnp.float16
    speech_path = 'data/train_mp3s/'
    data_path = 'data/train.csv'
    num_workers = 0
    output_path = ''
    background = "/home/zhenlan/Desktop/Projects/Bengali ASR/ESC-50-master/audio"
    add_data_path2 = '/home/zhenlan/Desktop/Projects/Bengali ASR/text'
    add_audio2 = '/home/zhenlan/Desktop/Projects/Bengali ASR/RESPIN/dev/'

In [2]:
#from whisper_jax import FlaxWhisperForConditionalGeneration
from transformers import FlaxWhisperForConditionalGeneration
from functions import *
from functools import partial
import optax
from jax import random
from transformers import AutoTokenizer
import time
import warnings
warnings.filterwarnings('ignore')

/home/zhenlanwang0426/.venv311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pad_to_multiple_of = 1
max_length_gen = 48
epochs = 1
verbose = 5


# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="bn", task="transcribe")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
tokenizer.bos_token = tokenizer.bos_token_id = None
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
text = pd.read_csv(data_path)
add_data = pd.read_csv(add_data_path)
# data source 2
df = pd.read_csv(add_data_path2,names=['code'])
df[['code','sentence']] = df["code"].str.split(" ", n=1, expand=True)
df.sentence = df.sentence.str.strip()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [4]:
"https://github.com/karoldvl/ESC-50/archive/master.zip"
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    GainTransition,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
    )

# define augmentation
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=1, leave_length_unchanged=False),
        Gain(min_gain_db=-6, max_gain_db=6, p=1),
        GainTransition(min_gain_db=-6, max_gain_db=6),
        PitchShift(min_semitones=-4, max_semitones=4, p=1),
        AddBackgroundNoise(sounds_path=background, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0),
        AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=1),
    ]
)


In [5]:
dataset1 = AudioDataset(text,speech_path,lambda x:x.id+'.mp3',\
                       augmentation,orig_sr=32000, target_sr=16000)
dataset2 = AudioDataset(add_data,add_audio,\
                        lambda x:x.path,augmentation,orig_sr=48000, target_sr=16000)
dataset3 = AudioDataset(df,add_audio2,\
                        lambda x:x.code.split('_')[-1] + '.wav',augmentation,orig_sr=16000, target_sr=16000)
dataset = Add2Data(dataset1,dataset2)
dataset = Add2Data(dataset,dataset3)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, \
                        collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))

In [6]:
# audio,input_ids,attention_mask = next(iter(train_loader))
# audio.shape,input_ids.shape
# audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)

In [7]:
# load the processor and model
model, params = FlaxWhisperForConditionalGeneration.from_pretrained(
    model_path, dtype=dtype, _do_init=False)
model.config.forced_decoder_ids = None
model.config.bos_token_id = None
model.config.suppress_tokens = None
model.config.decoder_start_token_id = None
model.generation_config.decoder_start_token_id = [50258, 50302, 50359, 50363]# '<|startoftranscript|><|bn|><|transcribe|><|notimestamps|>
model.generation_config.forced_decoder_ids = None
"""A list of pairs of integers which indicates a mapping from generation indices to token indices 
that will be forced before sampling. For example, [[0, 123]] means the first generated token 
will always be a token of index 123."""
model.generation_config.suppress_tokens = None
model.generation_config.begin_suppress_tokens = None
model.generation_config.bos_token_id = None

tcmalloc: large alloc 1126498304 bytes == 0x9b2f0000 @  0x7f8bdd6cd680 0x7f8bdd6ee824 0x5e06a0 0x6381fd 0x6785d1 0x5e72b4 0x53724e 0x67775e 0x6258ab 0x635d05 0x58b1da 0x6da3bf 0x6da647 0x6dc2b3 0x58e8d5 0x70b63c 0x63f58e 0x58dffe 0x70b63c 0x63f58e 0x58dffe 0x70b63c 0x70b8d7 0x6251f7 0x635d05 0x58b1da 0x5644be 0x6316c3 0x633f31 0x58ed6d 0x70b63c


In [8]:
# params['lm_head'] = {'kernel':params['model']['decoder']['embed_tokens']['embedding'].T,\
#                      'bias'  :jnp.zeros(model.config.vocab_size,params['model']['decoder']['embed_tokens']['embedding'].dtype)}

In [9]:
import pickle
opt = optax.adamw(learning_rate=4e-5,
                  b1=0.9,
                  b2=0.98,
                  eps=1e-6,
                  weight_decay=1e-1,)#1e-1
opt = optax.chain(optax.clip_by_global_norm(4e-3),opt)
#opt_states = opt.init(embedding)
# opt_states = opt.init(params)
filehandler = open(output_path+"opt_states","rb")
opt_states = pickle.load(filehandler)
filehandler.close()

In [10]:
# @partial(jax.pmap,axis_name='data',in_axes=(None,None,0,0,0,None),out_axes=(None,None,None))
# def train_one_step_embed(embedding,params,audio,input_ids,attention_mask,opt_states):
#     def loss_fn(embedding,params,audio,input_ids,attention_mask):
#         params['model']['decoder']['embed_tokens']['embedding'] = embedding
#         out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=True).logits # (B, L, d)
#         return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], input_ids[:,4:])*attention_mask[:,4:])
#     grad_fn = jax.value_and_grad(loss_fn,has_aux=False)
#     out = grad_fn(embedding,params,audio,input_ids,attention_mask)
#     l,grads = jax.lax.pmean(out,'data')
#     updates, opt_states = opt.update(grads, opt_states,params=embedding)
#     embedding = optax.apply_updates(embedding, updates)
#     return embedding,opt_states,l

@partial(jax.pmap,axis_name='data')
def train_one_step(params,audio,input_ids,attention_mask,opt_states):
    def loss_fn(params,audio,input_ids,attention_mask):
        out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=True).logits # (B, L, d)
        return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], input_ids[:,4:])*attention_mask[:,4:])
    grad_fn = jax.value_and_grad(loss_fn,has_aux=False)
    out = grad_fn(params,audio,input_ids,attention_mask)
    l,grads = jax.lax.pmean(out,'data')
    updates, opt_states = opt.update(grads, opt_states,params=params)
    params = optax.apply_updates(params, updates)
    return params,opt_states,l

# @jax.jit
# def train_one_step(params,audio,input_ids,attention_mask,opt_states):
#     def loss_fn(params,audio,input_ids,attention_mask):
#         out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=True).logits # (B, L, d)
#         return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], input_ids[:,4:])*attention_mask[:,4:])
#     grad_fn = jax.value_and_grad(loss_fn,has_aux=False)
#     l,grads = grad_fn(params,audio,input_ids,attention_mask)
#     updates, opt_states = opt.update(grads, opt_states,params=params)
#     params = optax.apply_updates(params, updates)
#     return params,opt_states,l

# @jax.jit
# def train_one_step_embed(embedding,params,audio,input_ids,attention_mask,opt_states):
#     def loss_fn(embedding,params,audio,input_ids,attention_mask):
#         params['model']['decoder']['embed_tokens']['embedding'] = embedding
#         out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=True).logits # (B, L, d)
#         return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], multiple hosts are needed, and some finesse is required to manage them.input_ids[:,4:])*attention_mask[:,4:])
#     grad_fn = jax.value_and_grad(loss_fn,has_aux=False)
#     l,grads = grad_fn(embedding,params,audio,input_ids,attention_mask)
#     updates, opt_states = opt.update(grads, opt_states,params=embedding)
#     embedding = optax.apply_updates(embedding, updates)
#     return embedding,opt_states,l

# @jax.jit
# def eval_one_step(params,audio,input_ids,attention_mask):
#     out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=False).logits # (B, L, d)
#     return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], input_ids[:,4:])*attention_mask[:,4:])


# @jax.jit
# def generate(params,audio):
#     return model.generate(audio,params=params,max_length=max_length_gen)

# metric = evaluate.load("wer")
# def metric_one_step(params,audio,txt):
#     generated_ids = model.generate(audio,params=params,max_length=max_length_gen, num_beams=1, do_sample=False).sequences
#     transcriptions = tokenizer.batch_decode(generated_ids.tolist(), skip_special_tokens=True)
#     wer = metric.compute(predictions=transcriptions, references=txt)
#     return wer

# def batch_generate(loader):
#     pass
#     #transcriptions = [txt + "|" for txt in transcriptions]

In [ ]:
import logging
logging.basicConfig(filename='training.log', filemode='w', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

devices = jax.devices()
replicated_params = jax.device_put_replicated(params, devices)
replicated_opt_states = jax.device_put_replicated(opt_states, devices)
start = time.time()
for i in range(epochs):
    # train
    train_loss = 0
    for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
#        audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
#         embedding,opt_states,l = train_one_step_embed(embedding,params,audio,input_ids,attention_mask,opt_states)
        replicated_params,replicated_opt_states,l = train_one_step(replicated_params,audio,input_ids,attention_mask,replicated_opt_states)
        train_loss += l[0].item()
        if j>0 and j%verbose == 0:
            train_loss /= verbose
            logging.info(f"iterations:{j}, loss: {train_loss:.3f}")
            train_loss = 0
            if j%25 ==0:
                model.save_pretrained(output_path+'model_all',jax.tree_map(lambda x: x[0], replicated_params))
                filehandler = open(output_path+"opt_states","wb")
                pickle.dump(jax.tree_map(lambda x: x[0], replicated_opt_states),filehandler)
                filehandler.close()
                
            if (time.time() - start)/60/60 > 24: # 4 hours
                break
        # # eval
        # if j%verbose == 0:
        #     train_loss /= verbose    
        #     eval_metric = 0
        #     params['model']['decoder']['embed_tokens']['embedding'] = embedding
        #     for k,(audio,input_ids,attention_mask) in enumerate(test_loader):
        #         audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
        #         eval_l = eval_one_step(params,audio,input_ids,attention_mask)
        #         eval_metric += eval_l.item()
            
        #     eval_metric /= k
        #     print(f"iterations:{j}, loss: {train_loss:.3f}, wer: {eval_metric:.3f}")
        #     train_loss = 0

iterations:5, loss: 1.733
iterations:10, loss: 1.502
iterations:15, loss: 1.552
iterations:20, loss: 1.644


In [ ]:
model.save_pretrained(output_path+'model_all',jax.tree_map(lambda x: x[0], replicated_params))
filehandler = open(output_path+"opt_states","wb")
pickle.dump(jax.tree_map(lambda x: x[0], replicated_opt_states),filehandler)
filehandler.close()